In [ ]:
добавить моду в myinfo

In [ ]:
добавить общее количество nan в myinfo

In [1]:
# ipynb jptr jupyter import imports head

# crucial imports
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from scipy import stats
%matplotlib inline

# pandas options
pd.set_option('display.max_columns', None)
pd.set_option('precision', 4)
pd.set_option("display.precision", 4)
pd.set_option("display.max_rows", 100)
pd.set_option("display.min_rows", 100)
pd.set_option("display.max_colwidth", 150)
pd.options.mode.chained_assignment = None

# visuals
# from jupyterthemes import jtplot
# jtplot.style(theme='onedork')
from IPython.core.display import display, HTML

display(
    HTML(
        """<style> 

        .container 
        { width:95% !important;}

        .rendered_html table, 
        .rendered_html th, 
        .rendered_html tr,
        .rendered_html td
        { border: 2px #535754 solid !important; 
          text-align: center;}

        .dataframe thead tr:only-child th {
            text-align: center;
        }
        .dataframe thead th {
            text-align: center;
        }

        .dataframe tbody tr th:only-of-type {
            vertical-align: middle;
        }
        .dataframe tbody tr th {
            vertical-align: top;
        }

        * { font-variant-ligatures: none; }

         </style>"""
    )
)

# my added functions
def my_info(dataframe):
    myinfo_colnames = ['Column', 'Data Type', 'NaN Count', 'NaN Percentage', 'Mean', 'Minimum', 'Maximum', 'Median']
    src = list(dataframe.columns)
    out = pd.DataFrame(src)
    myinfo_dtypes = pd.DataFrame(list(dataframe.dtypes))

    nclist = list(dataframe[col].isna().sum() for col in src)
    myinfo_nan_counts = pd.DataFrame(nclist)
    myinfo_nan_pct = pd.DataFrame(list(str(round(i/len(dataframe)*100,2))+'%' for i in nclist))

    dttps_lst = list(dataframe.dtypes)

    non_number_indexes = []
    for i in range(len(dttps_lst)):
        if dttps_lst[i] != 'float64' and dttps_lst[i] != 'int64' or dttps_lst[i] == 'datetime64':
            non_number_indexes.append(i)

    non_number_indexes.reverse()
    for i in non_number_indexes:
        src.pop(i)

    non_number_indexes.reverse()

    meanlist = list(np.nanmean(dataframe[col]) for col in src)
    for i in non_number_indexes:
        meanlist.insert(i, 'NaN')
    minlist = list(np.min(dataframe[col]) for col in src)
    for i in non_number_indexes:
        minlist.insert(i, 'NaN')
    maxlist = list(np.max(dataframe[col]) for col in src)
    for i in non_number_indexes:
        maxlist.insert(i, 'NaN')
    medianlist = list(np.nanmedian(dataframe[col]) for col in src)
    for i in non_number_indexes:
        medianlist.insert(i, 'NaN')

    meanlist = pd.DataFrame(meanlist)
    minlist = pd.DataFrame(minlist)
    maxlist = pd.DataFrame(maxlist)
    medianlist = pd.DataFrame(medianlist)

    out = pd.concat([ out, myinfo_dtypes ], axis=1)
    out = pd.concat([ out, myinfo_nan_counts ], axis=1)
    out = pd.concat([ out, myinfo_nan_pct ], axis=1)

    out = pd.concat([ out, meanlist ], axis=1)
    out = pd.concat([ out, minlist ], axis=1)
    out = pd.concat([ out, maxlist ], axis=1)
    out = pd.concat([ out, medianlist ], axis=1)

    out.columns = myinfo_colnames
    return out

def scale_column_zero_to_one(column):
    try:
        if column.max() == 1 and column.min() == 0:
            return column
        else:
            yy = column/column.max()
            return yy
    except:
        return column

def my_heatmap(dataframe):
    plt.figure(figsize=(200,80))
    ax=sns.heatmap(dataframe.corr(),annot=True)

def my_countplot(*args, **kwargs):
    plt.figure(figsize=(20,8))
    ax = sns.countplot(**kwargs)
    total = 0
    for p in ax.patches:
        total += p.get_height()
    for p in ax.patches:
        percentage = round(100*p.get_height()/total,2)
        ax.annotate(
            f'\n{p.get_height()} values, {percentage}%', 
            (p.get_x() + p.get_width() / 2, p.get_height()), 
            ha='center', va='top', color='white', size=22//(len(ax.patches)/4))

def my_histogram_of_col(col):
    fig, ax = plt.subplots(figsize=(20,8))
    sns.histplot(col, color='skyblue', stat='density')
    sns.kdeplot(col, color='orange')

def csv_export(dataframe):
    temp = str(datetime.datetime.now()).replace('.',' ').replace(':','_').replace('-','_')
    temp = temp.split()
    del temp[-1]
    curdt='_'.join(temp)
    varname = [name for name in globals() if globals()[name] is dataframe][0]
    filename = varname+'_'+curdt+'.csv'
    dataframe.to_csv(filename,index=False)

def aa(inp=''):
    try:
        print([name for name in globals() if globals()[name] is inp][0], end='')
    except:
        pass
    try:
        display(inp)
    except:
        print(inp)


In [2]:
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm

# print("Mean absolute error =", round(sm.mean_absolute_error(test_label, preds_official), 2)) 
# print("Mean squared error =", round(sm.mean_squared_error(test_label, preds_official), 2)) 
# print("Median absolute error =", round(sm.median_absolute_error(test_label, preds_official), 2)) 
# print("Explain variance score =", round(sm.explained_variance_score(test_label, preds_official), 2)) 
# print("R2 score =", round(sm.r2_score(test_label, preds), 2))

In [3]:
def check_for_ints(val):
    try:
        return int(val.split('.')[0])
    except:
        return np.nan

In [4]:
def objinfo(df):
    polinfo = my_info(df)
    return polinfo[polinfo['Data Type']=='object']
    

In [5]:
def floor_and_nans_fix(polidf):
    polinfo = my_info(polidf)
    print('polinfo')
    aa(polinfo)
    polidf['floor'] = polidf['floor'].apply(lambda x: check_for_ints(x))
    cols_to_fill = list(polinfo[polinfo['NaN Count']>0]['Column'])
    for col in cols_to_fill:
        print(col)
        print(polidf[col].mode()[0])
        polidf[col] = polidf[col].fillna(polidf[col].mode()[0])
    return polidf

In [6]:
test_official = pd.read_csv('test.csv')
test_official

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type
0,Курск,1.0,COL_289284,7,55,85,117,0,0,0,0,0,2,6,10,4.1017,Курск,4.4321e+05,0.1471,0,7,7,19,0,0,0,0,1,2,3,3,5,5,5,2,2,4,7,7,8,0,0,1,0,0,0,1,5,36,61,83,435.3635,5.6368,0,0,0,0.1087,11,15,27,184,78,1997.0,743.0,4.3250,4.2113,1966.4716,1966.7403,Курская область,51.7093,36.1479,156.1490,S6983,2020-09-06,100,1
1,Сургут,1.0,COL_289305,8,70,112,140,0,0,0,0,0,7,8,13,4.0842,Сургут,3.8063e+05,0.1625,0,18,33,50,0,0,0,1,1,2,5,6,3,6,8,1,1,1,3,5,5,3,6,7,0,3,5,8,4,51,81,99,693.4479,9.6416,0,0,0,0.0914,9,11,19,118,44,2823.0,1019.0,5.3898,5.5000,1988.2593,1989.0682,Ханты-Мансийский АО,61.2332,73.4625,190.7379,S29120,2020-09-06,110,1
2,Тюмень,-1.0,COL_289318,3,28,67,122,0,0,0,0,0,9,18,34,1.2920,Тюмень,7.4455e+05,0.0758,1,21,57,92,0,0,1,2,0,1,4,5,2,3,3,1,2,5,0,1,1,0,2,5,0,1,1,3,3,15,34,63,298.9047,1.8040,0,0,0,0.1203,6,20,29,150,51,3484.0,1332.0,7.9155,8.2500,1985.8803,1991.4583,Тюменская область,57.1431,65.5546,457.1181,S23731,2020-09-06,10,1
3,Иркутск,1.0,COL_289354,5,76,139,231,0,0,0,0,0,28,52,84,0.9146,Иркутск,6.2356e+05,0.0436,1,30,60,95,1,8,13,18,0,5,6,10,5,8,12,12,20,32,1,8,18,1,5,5,0,7,11,16,3,14,26,51,60.2247,1.3381,0,0,0,0.0793,9,18,29,252,86,1975.0,666.0,3.2769,3.0120,1947.0733,1941.6579,Иркутская область,52.2814,104.2830,66.5036,S14207,2020-09-06,100,1
4,Курск,NaN,COL_289399,8,105,189,279,0,0,2,9,1,24,40,54,1.3001,Курск,4.4321e+05,0.0842,1,40,72,84,1,10,12,12,3,10,14,21,2,6,6,10,14,17,1,1,1,4,5,12,0,2,12,16,1,49,98,156,432.5301,2.4410,0,0,0,0.0936,12,26,37,114,31,1044.0,394.0,4.3462,4.8276,1948.7642,1946.6897,Курская область,51.7297,36.1940,23.8649,S20658,2020-09-06,10,1
5,Иркутск,1.0,COL_289428,0,18,30,33,0,0,0,0,0,6,9,9,5.4943,Иркутск,6.2356e+05,0.1410,0,13,15,17,0,0,0,0,0,0,0,0,2,3,3,0,0,0,1,2,2,0,0,0,0,2,2,2,0,8,14,16,58.4055,6.7825,0,0,0,0.1867,7,12,12,62,27,489.0,260.0,2.9344,3.1852,1963.6066,1968.9231,Иркутская область,52.2763,104.3612,58.3778,S14771,2020-09-06,10,1
6,Кемерово,NaN,COL_289460,1,5,7,20,0,0,0,0,0,0,0,0,6.4769,Кемерово,5.5866e+05,0.5237,0,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,1,5,5,12,206.3094,3.4572,0,

In [26]:
df = pd.read_csv('train.csv', low_memory=False)
df = df.replace('饶河镇', 'Жаохэ')
df = df.replace('绥芬河市', 'Суйфыньхэ')
df = df[df['price_type']==1]

In [27]:
def prep_for_prediction(df, istrain=True):
    df = df.drop(columns=['street', 'date', 'lat', 'lng'])
    
    # перед тем как заполнить модой по регионам
    # распарсим данные насколько это возможно
    df.floor.unique()

    # добавляю столбец, в котором будет храниться количество занимаемых этажей
    df['floors'] = 0

    # первый шаг: перевожу в инт те этажи, которые выглядят как "3.0",
    # в количество занимаемых этажей записываю 1
    # а по остальным записываю их индекс, чтобы рабоать с нимим дальше
    idx_to_fix = []
    for floor in df.index:
        try:
            df.loc[floor, 'floor'] = int(df.loc[floor, 'floor'].split('.')[0])
            df.loc[floor, 'floors'] = 1
        except:
            idx_to_fix.append(floor)


    # встречаемые символы и слова в строках 
    possible_del = ['(', '+', '-', ')']
    uniq_ = ['антресоль', 'мансарда', 'мезонин', 'подва', 'подвал', 'тех.этаж', 'техэтаж', 'цоколь', 'цокольный', 'Техническое подполье', 'Техническое', 'подполье']

    # второй шаг: 
    for row in idx_to_fix:
        for word in str(df.loc[row, 'floor']).replace(',', ' ').split(' '):
            floors = []
            count = 0
            # print(word)
            try:  
                if '-' in word:
                    fl = word.replace('-', ' ').split(' ')
                    try:
                        if len(fl) >= 2:
                            for f in range(int(fl[0]), int(fl[1])+1):
                                floors.append(f)
                    except IndexError:
                        floors.append(int(fl[0]))
                elif '(' in word:
                    fl = word.replace('(', ' ').split(' ')
                    floors.append(int(fl[0]))
                else:
                    floors.append(int(word))
            except ValueError:
                if word.lower() in uniq_:
                    df.loc[row, 'floors'] += 1

            if floors != []:
                df.loc[row, 'floors'] += len(floors)
                df.loc[row, 'floor'] = min(floors)

    # третий шаг: 
    one_more_go = []
    for idx in idx_to_fix:
        try:
            int(df.loc[idx, 'floor'])
        except ValueError:
            one_more_go.append(idx)

    for row in one_more_go:
        for word in str(df.loc[row, 'floor']).replace(', ', ' ').split(' '):
            floors = []
            if word.lower().replace('.', '') in uniq_:
                df.loc[row, 'floor'] = 1
            elif '-' in word:
                fl = word.replace('-', ' ').split(' ')
                try:
                    if len(fl) >= 2:
                        for f in range(int(fl[0]), int(fl[1])+1):
                            floors.append(f)
                except ValueError:
                    floors.append(int(fl[0]))
                df.loc[row, 'floor'] = min(floors)
                df.loc[row, 'floors'] += len(floors)

    floor_mode_by_region = df.groupby(['region']).floor.apply(lambda x: x.mode()).reset_index(0).set_index('region')
    floor_mode_by_region
    for idx in df.loc[df.floor.isna()].index:
        try:
            df.loc[idx, 'floor'] = int(floor_mode_by_region.loc[df.loc[idx, 'region']])
        except:
            df.loc[idx, 'floor'] = int(df.floor.mode())


    df.loc[df['floors'] == 0, 'floors'] = 1
    # заполняем пропуски в остальных столбцах 
    for col in pd.DataFrame(df.isna().sum(), columns=['%nan']).loc[df.isna().sum() != 0].index:
        for idx in df[np.isnan(df[col])].index:
            df.loc[idx, col] = df.groupby('region')[col].mean()[df.loc[idx, 'region']]
    
    # оставшиеся удаляем 
    for col in pd.DataFrame((df.isna().sum() / df.shape[0])*100, columns=['%nan']).loc[df.isna().sum() != 0].index:
        df[col].fillna(df[col].mean(), inplace=True)
    
    ## ЭТО ВСТАВИТЬ 
    df['osm_amenity_points_in_0.005'] = df['osm_amenity_points_in_0.005'].sub(df['osm_amenity_points_in_0.001'], axis=0)
    df['osm_amenity_points_in_0.0075'] = df['osm_amenity_points_in_0.0075'].sub(df['osm_amenity_points_in_0.005'], axis=0).sub(df['osm_amenity_points_in_0.001'], axis=0)
    df['osm_amenity_points_in_0.01'] = df['osm_amenity_points_in_0.01'].sub(df['osm_amenity_points_in_0.0075'], axis=0).sub(df['osm_amenity_points_in_0.005'], axis=0).sub(df['osm_amenity_points_in_0.001'], axis=0)

    df['osm_amenity_points_in_0.001'] = df['osm_amenity_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_amenity_points_in_0.005'] = df['osm_amenity_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_amenity_points_in_0.0075'] = df['osm_amenity_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_amenity_points_in_0.01'] = df['osm_amenity_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_amenity_points'] = (df['osm_amenity_points_in_0.001'] + df['osm_amenity_points_in_0.01'] + df['osm_amenity_points_in_0.0075'] + df['osm_amenity_points_in_0.005']) / 4

    # ----
    df['osm_building_points_in_0.005'] = df['osm_building_points_in_0.005'].sub(df['osm_building_points_in_0.001'], axis=0)
    df['osm_building_points_in_0.0075'] = df['osm_building_points_in_0.0075'].sub(df['osm_building_points_in_0.005'], axis=0).sub(df['osm_building_points_in_0.001'], axis=0)
    df['osm_building_points_in_0.01'] = df['osm_building_points_in_0.01'].sub(df['osm_building_points_in_0.0075'], axis=0).sub(df['osm_building_points_in_0.005'], axis=0).sub(df['osm_building_points_in_0.001'], axis=0)

    df['osm_building_points_in_0.001'] = df['osm_building_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_building_points_in_0.005'] = df['osm_building_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_building_points_in_0.0075'] = df['osm_building_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_building_points_in_0.01'] = df['osm_building_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_building_points'] = (df['osm_building_points_in_0.001']+ df['osm_building_points_in_0.005'] + df['osm_building_points_in_0.0075'] + df['osm_building_points_in_0.01']) / 4

    # ----

    df['osm_crossing_points_in_0.005'] = df['osm_crossing_points_in_0.005'].sub(df['osm_crossing_points_in_0.001'], axis=0)
    df['osm_crossing_points_in_0.0075'] = df['osm_crossing_points_in_0.0075'].sub(df['osm_crossing_points_in_0.005'], axis=0).sub(df['osm_crossing_points_in_0.001'], axis=0)
    df['osm_crossing_points_in_0.01'] = df['osm_crossing_points_in_0.01'].sub(df['osm_crossing_points_in_0.0075'], axis=0).sub(df['osm_crossing_points_in_0.005'], axis=0).sub(df['osm_crossing_points_in_0.001'], axis=0)

    df['osm_crossing_points_in_0.001'] = df['osm_crossing_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_crossing_points_in_0.005'] = df['osm_crossing_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_crossing_points_in_0.0075'] = df['osm_crossing_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_crossing_points_in_0.01'] = df['osm_crossing_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_crossing_points'] = (df['osm_crossing_points_in_0.001']+ df['osm_crossing_points_in_0.005'] + df['osm_crossing_points_in_0.0075'] + df['osm_crossing_points_in_0.01']) / 4

    #----
    df['osm_catering_points_in_0.005'] = df['osm_catering_points_in_0.005'].sub(df['osm_catering_points_in_0.001'], axis=0)
    df['osm_catering_points_in_0.0075'] = df['osm_catering_points_in_0.0075'].sub(df['osm_catering_points_in_0.005'], axis=0).sub(df['osm_catering_points_in_0.001'], axis=0)
    df['osm_catering_points_in_0.01'] = df['osm_catering_points_in_0.01'].sub(df['osm_catering_points_in_0.0075'], axis=0).sub(df['osm_catering_points_in_0.005'], axis=0).sub(df['osm_catering_points_in_0.001'], axis=0)

    df['osm_catering_points_in_0.001'] = df['osm_catering_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_catering_points_in_0.005'] = df['osm_catering_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_catering_points_in_0.0075'] = df['osm_catering_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_catering_points_in_0.01'] = df['osm_catering_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_catering_points'] = (df['osm_catering_points_in_0.001']+ df['osm_catering_points_in_0.005'] + df['osm_catering_points_in_0.0075'] + df['osm_catering_points_in_0.01']) / 4


    #----
    df['osm_culture_points_in_0.005'] = df['osm_culture_points_in_0.005'].sub(df['osm_culture_points_in_0.001'], axis=0)
    df['osm_culture_points_in_0.0075'] = df['osm_culture_points_in_0.0075'].sub(df['osm_culture_points_in_0.005'], axis=0).sub(df['osm_culture_points_in_0.001'], axis=0)
    df['osm_culture_points_in_0.01'] = df['osm_culture_points_in_0.01'].sub(df['osm_culture_points_in_0.0075'], axis=0).sub(df['osm_culture_points_in_0.005'], axis=0).sub(df['osm_culture_points_in_0.001'], axis=0)

    df['osm_culture_points_in_0.001'] = df['osm_culture_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_culture_points_in_0.005'] = df['osm_culture_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_culture_points_in_0.0075'] = df['osm_culture_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_culture_points_in_0.01'] = df['osm_culture_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_culture_points'] = (df['osm_culture_points_in_0.001']+ df['osm_culture_points_in_0.005'] + df['osm_culture_points_in_0.0075'] + df['osm_culture_points_in_0.01']) / 4

    #----
    df['osm_finance_points_in_0.005'] = df['osm_finance_points_in_0.005'].sub(df['osm_finance_points_in_0.001'], axis=0)
    df['osm_finance_points_in_0.0075'] = df['osm_finance_points_in_0.0075'].sub(df['osm_finance_points_in_0.005'], axis=0).sub(df['osm_finance_points_in_0.001'], axis=0)
    df['osm_finance_points_in_0.01'] = df['osm_finance_points_in_0.01'].sub(df['osm_finance_points_in_0.0075'], axis=0).sub(df['osm_finance_points_in_0.005'], axis=0).sub(df['osm_finance_points_in_0.001'], axis=0)

    df['osm_finance_points_in_0.001'] = df['osm_finance_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_finance_points_in_0.005'] = df['osm_finance_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_finance_points_in_0.0075'] = df['osm_finance_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_finance_points_in_0.01'] = df['osm_finance_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_finance_points'] = (df['osm_finance_points_in_0.001']+ df['osm_finance_points_in_0.005'] + df['osm_finance_points_in_0.0075'] + df['osm_finance_points_in_0.01']) / 4

    #----
    df['osm_healthcare_points_in_0.0075'] = df['osm_healthcare_points_in_0.0075'].sub(df['osm_healthcare_points_in_0.005'], axis=0)
    df['osm_healthcare_points_in_0.01'] = df['osm_healthcare_points_in_0.01'].sub(df['osm_healthcare_points_in_0.0075'], axis=0).sub(df['osm_healthcare_points_in_0.005'], axis=0)


    df['osm_healthcare_points_in_0.005'] = df['osm_healthcare_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_healthcare_points_in_0.0075'] = df['osm_healthcare_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_healthcare_points_in_0.01'] = df['osm_healthcare_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_healthcare_points'] = (df['osm_healthcare_points_in_0.005'] + df['osm_healthcare_points_in_0.0075'] + df['osm_healthcare_points_in_0.01']) / 3

    #----
    df['osm_historic_points_in_0.0075'] = df['osm_historic_points_in_0.0075'].sub(df['osm_historic_points_in_0.005'], axis=0)
    df['osm_historic_points_in_0.01'] = df['osm_historic_points_in_0.01'].sub(df['osm_historic_points_in_0.0075'], axis=0).sub(df['osm_historic_points_in_0.005'], axis=0)

    df['osm_historic_points_in_0.005'] = df['osm_historic_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_historic_points_in_0.0075'] = df['osm_historic_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_historic_points_in_0.01'] = df['osm_historic_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_historic_points'] = (df['osm_historic_points_in_0.005'] + df['osm_historic_points_in_0.0075'] + df['osm_historic_points_in_0.01']) / 3

    #----
    df['osm_hotels_points_in_0.0075'] = df['osm_hotels_points_in_0.0075'].sub(df['osm_hotels_points_in_0.005'], axis=0)
    df['osm_hotels_points_in_0.01'] = df['osm_hotels_points_in_0.01'].sub(df['osm_hotels_points_in_0.0075'], axis=0).sub(df['osm_hotels_points_in_0.005'], axis=0)

    df['osm_hotels_points_in_0.005'] = df['osm_hotels_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_hotels_points_in_0.0075'] = df['osm_hotels_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_hotels_points_in_0.01'] = df['osm_hotels_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_hotels_points'] = ( df['osm_hotels_points_in_0.005'] + df['osm_hotels_points_in_0.0075'] + df['osm_hotels_points_in_0.01']) / 3

    #----
    df['osm_leisure_points_in_0.0075'] = df['osm_leisure_points_in_0.0075'].sub(df['osm_leisure_points_in_0.005'], axis=0)
    df['osm_leisure_points_in_0.01'] = df['osm_leisure_points_in_0.01'].sub(df['osm_leisure_points_in_0.0075'], axis=0).sub(df['osm_leisure_points_in_0.005'], axis=0)


    df['osm_leisure_points_in_0.005'] = df['osm_leisure_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_leisure_points_in_0.0075'] = df['osm_leisure_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_leisure_points_in_0.01'] = df['osm_leisure_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_catering_points'] = (df['osm_catering_points_in_0.005'] + df['osm_catering_points_in_0.0075'] + df['osm_catering_points_in_0.01']) / 3

    #----
    df['osm_offices_points_in_0.005'] = df['osm_offices_points_in_0.005'].sub(df['osm_offices_points_in_0.001'], axis=0)
    df['osm_offices_points_in_0.0075'] = df['osm_offices_points_in_0.0075'].sub(df['osm_offices_points_in_0.005'], axis=0).sub(df['osm_offices_points_in_0.001'], axis=0)
    df['osm_offices_points_in_0.01'] = df['osm_offices_points_in_0.01'].sub(df['osm_offices_points_in_0.0075'], axis=0).sub(df['osm_offices_points_in_0.005'], axis=0).sub(df['osm_offices_points_in_0.001'], axis=0)

    df['osm_offices_points_in_0.001'] = df['osm_offices_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_offices_points_in_0.005'] = df['osm_offices_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_offices_points_in_0.0075'] = df['osm_offices_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_offices_points_in_0.01'] = df['osm_offices_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_offices_points'] = (df['osm_offices_points_in_0.001']+ df['osm_offices_points_in_0.005'] + df['osm_offices_points_in_0.0075'] + df['osm_offices_points_in_0.01']) / 4

    #----
    df['osm_shops_points_in_0.005'] = df['osm_shops_points_in_0.005'].sub(df['osm_shops_points_in_0.001'], axis=0)
    df['osm_shops_points_in_0.0075'] = df['osm_shops_points_in_0.0075'].sub(df['osm_shops_points_in_0.005'], axis=0).sub(df['osm_shops_points_in_0.001'], axis=0)
    df['osm_shops_points_in_0.01'] = df['osm_shops_points_in_0.01'].sub(df['osm_shops_points_in_0.0075'], axis=0).sub(df['osm_shops_points_in_0.005'], axis=0).sub(df['osm_shops_points_in_0.001'], axis=0)

    df['osm_shops_points_in_0.001'] = df['osm_shops_points_in_0.001'] /  (np.pi * pow(100, 2))
    df['osm_shops_points_in_0.005'] = df['osm_shops_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_shops_points_in_0.0075'] = df['osm_shops_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_shops_points_in_0.01'] = df['osm_shops_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_shops_points'] = (df['osm_shops_points_in_0.001']+ df['osm_shops_points_in_0.005'] + df['osm_shops_points_in_0.0075'] + df['osm_shops_points_in_0.01']) / 4

    #----
    df['osm_train_stop_points_in_0.0075'] = df['osm_train_stop_points_in_0.0075'].sub(df['osm_train_stop_points_in_0.005'], axis=0)
    df['osm_train_stop_points_in_0.01'] = df['osm_train_stop_points_in_0.01'].sub(df['osm_train_stop_points_in_0.0075'], axis=0).sub(df['osm_train_stop_points_in_0.005'], axis=0)

    df['osm_train_stop_points_in_0.005'] = df['osm_train_stop_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_train_stop_points_in_0.0075'] = df['osm_train_stop_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_train_stop_points_in_0.01'] = df['osm_train_stop_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_train_points'] = (df['osm_train_stop_points_in_0.005'] + df['osm_train_stop_points_in_0.0075'] + df['osm_train_stop_points_in_0.01']) / 3

    #----
    df['osm_transport_stop_points_in_0.0075'] = df['osm_transport_stop_points_in_0.0075'].sub(df['osm_transport_stop_points_in_0.005'], axis=0)
    df['osm_transport_stop_points_in_0.01'] = df['osm_transport_stop_points_in_0.01'].sub(df['osm_transport_stop_points_in_0.0075'], axis=0).sub(df['osm_transport_stop_points_in_0.005'], axis=0)

    df['osm_transport_stop_points_in_0.005'] = df['osm_transport_stop_points_in_0.005'] / (np.pi * pow(500, 2))
    df['osm_transport_stop_points_in_0.0075'] = df['osm_transport_stop_points_in_0.0075'] / (np.pi * pow(750, 2))
    df['osm_transport_stop_points_in_0.01'] = df['osm_transport_stop_points_in_0.01'] / (np.pi * pow(1000, 2))

    df['mean_transport_points'] = (df['osm_transport_stop_points_in_0.005'] + df['osm_transport_stop_points_in_0.0075'] + df['osm_transport_stop_points_in_0.01']) / 3

    to_delete = ['osm_amenity_points_in_0.001','osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075','osm_amenity_points_in_0.01', 'osm_building_points_in_0.001','osm_building_points_in_0.005', 'osm_building_points_in_0.0075','osm_building_points_in_0.01', 'osm_catering_points_in_0.001','osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075','osm_catering_points_in_0.01', 'osm_city_closest_dist','osm_city_nearest_name', 'osm_city_nearest_population','osm_crossing_closest_dist', 'osm_crossing_points_in_0.001','osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075','osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001','osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075','osm_culture_points_in_0.01', 'osm_finance_points_in_0.001','osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075','osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005','osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01','osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075','osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005','osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01','osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075','osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001','osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075','osm_offices_points_in_0.01', 'osm_shops_points_in_0.001','osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075','osm_shops_points_in_0.01', 'osm_subway_closest_dist','osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005','osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01','osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005','osm_transport_stop_points_in_0.0075','osm_transport_stop_points_in_0.01']
    df.drop(to_delete, axis=1, inplace=True)
    df.drop(['floors', 'reform_mean_floor_count_1000', 'reform_mean_floor_count_500', 'price_type'], axis = 1, inplace=True)

    polinfo = my_info(df)
#     df = floor_and_nans_fix(df)
    if istrain:
        train_data, train_label = df.drop('per_square_meter_price', axis = 1), df['per_square_meter_price']
        return train_data, train_label
    else:
        return df

In [28]:
my_info(df)

,Column,Data Type,NaN Count,NaN Percentage,Mean,Minimum,Maximum,Median
0,city,object,0,0.0%,NaN,NaN,NaN,NaN
1,floor,object,2464,54.84%,NaN,NaN,NaN,NaN
2,id,object,0,0.0%,NaN,NaN,NaN,NaN
3,lat,float64,0,0.0%,55.494,42.7825,69.4966,55.8028
4,lng,float64,0,0.0%,62.9948,20.2266,133.3948,56.2912
5,osm_amenity_points_in_0.001,int64,0,0.0%,3.6608,0,34,2.0
6,osm_amenity_points_in_0.005,int64,0,0.0%,60.1347,0,434,33.0
7,osm_amenity_points_in_0.0075,int64,0,0.0%,123.2177,0,851,70.0
8,osm_amenity_points_in_0.01,int64,0,0.0%,200.0294,0,1378,110.0
9,osm_building_points_in_0.001,int64,0,0.0%,0.0178,0,2,0.0


In [29]:
train_data, train_label = prep_for_prediction(df)

In [30]:
my_info(train_data)

,Column,Data Type,NaN Count,NaN Percentage,Mean,Minimum,Maximum,Median
0,city,object,0,0.0%,NaN,NaN,NaN,NaN
1,floor,object,0,0.0%,NaN,NaN,NaN,NaN
2,id,object,0,0.0%,NaN,NaN,NaN,NaN
3,reform_count_of_houses_1000,int64,0,0.0%,155.8164,0,628,135.0
4,reform_count_of_houses_500,int64,0,0.0%,46.4138,0,217,41.0
5,reform_house_population_1000,float64,0,0.0%,2679.8885,0.0,8246.0,2463.0
6,reform_house_population_500,float64,0,0.0%,798.1678,0.0,2811.0,741.0
7,reform_mean_year_building_1000,float64,0,0.0%,1962.5166,1607.2695,2016.8889,1968.4779
8,reform_mean_year_building_500,float64,0,0.0%,1962.9429,1505.4146,2017.3333,1968.5692
9,region,object,0,0.0%,NaN,NaN,NaN,NaN


In [31]:
objinfo(train_data)

,Column,Data Type,NaN Count,NaN Percentage,Mean,Minimum,Maximum,Median
0,city,object,0,0.0%,NaN,NaN,NaN,NaN
1,floor,object,0,0.0%,NaN,NaN,NaN,NaN
2,id,object,0,0.0%,NaN,NaN,NaN,NaN
9,region,object,0,0.0%,NaN,NaN,NaN,NaN


In [32]:
def get_cat_vars(df):
    oinf = objinfo(df)
    cat_vars = [oinf[oinf['Column'] == i].index[0] for i in list(oinf['Column'])]
    return cat_vars

In [33]:
train_pool = Pool(train_data, 
                  train_label 
                  ,cat_features= get_cat_vars(train_data)
                 )

# specify the training parameters 
model = CatBoostRegressor(iterations=1500, 
                          depth=7, 
                          learning_rate=0.05, 
                          loss_function='MAE', ignored_features=[2])
#train the model
model.fit(train_pool)
# make the prediction using the resulting model


0:	learn: 31603.6470416	total: 10.7ms	remaining: 16.1s
1:	learn: 30915.6811689	total: 19.4ms	remaining: 14.5s
2:	learn: 30301.9758522	total: 31ms	remaining: 15.5s
3:	learn: 29750.7468039	total: 41.8ms	remaining: 15.6s
4:	learn: 29183.4732186	total: 50ms	remaining: 14.9s
5:	learn: 28660.0551994	total: 58.1ms	remaining: 14.5s
6:	learn: 28135.9894068	total: 68.5ms	remaining: 14.6s
7:	learn: 27695.7426660	total: 77ms	remaining: 14.4s
8:	learn: 27318.0091253	total: 89.3ms	remaining: 14.8s
9:	learn: 26957.0212495	total: 97.6ms	remaining: 14.5s
10:	learn: 26669.1413074	total: 108ms	remaining: 14.7s
11:	learn: 26328.4351273	total: 118ms	remaining: 14.6s
12:	learn: 26013.4617476	total: 127ms	remaining: 14.5s
13:	learn: 25702.0888939	total: 137ms	remaining: 14.5s
14:	learn: 25419.9682854	total: 145ms	remaining: 14.4s
15:	learn: 25144.2117977	total: 152ms	remaining: 14.1s
16:	learn: 24923.0897314	total: 162ms	remaining: 14.2s
17:	learn: 24674.3863260	total: 169ms	remaining: 13.9s
18:	learn: 24447

165:	learn: 17592.4143977	total: 1.83s	remaining: 14.7s
166:	learn: 17568.9082364	total: 1.85s	remaining: 14.7s
167:	learn: 17526.5785447	total: 1.86s	remaining: 14.7s
168:	learn: 17509.7466101	total: 1.87s	remaining: 14.7s
169:	learn: 17492.0083367	total: 1.88s	remaining: 14.7s
170:	learn: 17477.4670759	total: 1.9s	remaining: 14.7s
171:	learn: 17458.7566564	total: 1.91s	remaining: 14.8s
172:	learn: 17445.1932821	total: 1.92s	remaining: 14.8s
173:	learn: 17400.5738889	total: 1.94s	remaining: 14.8s
174:	learn: 17388.7381629	total: 1.95s	remaining: 14.7s
175:	learn: 17377.6755913	total: 1.95s	remaining: 14.7s
176:	learn: 17355.1009603	total: 1.96s	remaining: 14.7s
177:	learn: 17326.2050790	total: 1.97s	remaining: 14.6s
178:	learn: 17308.4932120	total: 1.99s	remaining: 14.7s
179:	learn: 17283.7177715	total: 2s	remaining: 14.6s
180:	learn: 17273.9867865	total: 2.01s	remaining: 14.6s
181:	learn: 17261.8608924	total: 2.01s	remaining: 14.6s
182:	learn: 17234.7375077	total: 2.02s	remaining: 14

327:	learn: 14829.5991836	total: 3.88s	remaining: 13.9s
328:	learn: 14819.0026236	total: 3.89s	remaining: 13.9s
329:	learn: 14799.4328095	total: 3.9s	remaining: 13.8s
330:	learn: 14774.9265017	total: 3.91s	remaining: 13.8s
331:	learn: 14767.0742290	total: 3.92s	remaining: 13.8s
332:	learn: 14758.5364796	total: 3.93s	remaining: 13.8s
333:	learn: 14754.1145642	total: 3.94s	remaining: 13.8s
334:	learn: 14750.2265723	total: 3.96s	remaining: 13.8s
335:	learn: 14745.3906317	total: 3.96s	remaining: 13.7s
336:	learn: 14739.6158045	total: 3.97s	remaining: 13.7s
337:	learn: 14727.7534569	total: 3.98s	remaining: 13.7s
338:	learn: 14718.4700193	total: 3.99s	remaining: 13.7s
339:	learn: 14714.3378400	total: 4s	remaining: 13.7s
340:	learn: 14703.8099556	total: 4.01s	remaining: 13.6s
341:	learn: 14686.5616588	total: 4.02s	remaining: 13.6s
342:	learn: 14663.9210989	total: 4.03s	remaining: 13.6s
343:	learn: 14645.7154458	total: 4.04s	remaining: 13.6s
344:	learn: 14637.0664597	total: 4.04s	remaining: 13

485:	learn: 13151.8458905	total: 5.91s	remaining: 12.3s
486:	learn: 13141.0795061	total: 5.92s	remaining: 12.3s
487:	learn: 13126.6508429	total: 5.93s	remaining: 12.3s
488:	learn: 13121.1507459	total: 5.94s	remaining: 12.3s
489:	learn: 13111.9013635	total: 5.95s	remaining: 12.3s
490:	learn: 13109.9065567	total: 5.96s	remaining: 12.2s
491:	learn: 13106.7255627	total: 5.97s	remaining: 12.2s
492:	learn: 13076.2474417	total: 5.98s	remaining: 12.2s
493:	learn: 13070.0426977	total: 5.99s	remaining: 12.2s
494:	learn: 13065.1351541	total: 6s	remaining: 12.2s
495:	learn: 13056.5339347	total: 6s	remaining: 12.2s
496:	learn: 13053.1323921	total: 6.01s	remaining: 12.1s
497:	learn: 13042.8942359	total: 6.02s	remaining: 12.1s
498:	learn: 13034.4200386	total: 6.03s	remaining: 12.1s
499:	learn: 13032.5286173	total: 6.04s	remaining: 12.1s
500:	learn: 13022.2679859	total: 6.05s	remaining: 12.1s
501:	learn: 13017.2077593	total: 6.06s	remaining: 12.1s
502:	learn: 13008.8632149	total: 6.08s	remaining: 12s


649:	learn: 12012.5602266	total: 7.9s	remaining: 10.3s
650:	learn: 12000.8315219	total: 7.91s	remaining: 10.3s
651:	learn: 12000.1579993	total: 7.92s	remaining: 10.3s
652:	learn: 11998.3744677	total: 7.93s	remaining: 10.3s
653:	learn: 11995.4657162	total: 7.93s	remaining: 10.3s
654:	learn: 11990.7531638	total: 7.94s	remaining: 10.2s
655:	learn: 11988.0484306	total: 7.95s	remaining: 10.2s
656:	learn: 11984.2496610	total: 7.97s	remaining: 10.2s
657:	learn: 11977.9717284	total: 7.97s	remaining: 10.2s
658:	learn: 11971.6870811	total: 7.99s	remaining: 10.2s
659:	learn: 11969.5975034	total: 7.99s	remaining: 10.2s
660:	learn: 11964.3190920	total: 8s	remaining: 10.2s
661:	learn: 11960.2249393	total: 8.01s	remaining: 10.1s
662:	learn: 11954.6300471	total: 8.02s	remaining: 10.1s
663:	learn: 11952.7417951	total: 8.03s	remaining: 10.1s
664:	learn: 11927.2234769	total: 8.03s	remaining: 10.1s
665:	learn: 11923.1739343	total: 8.04s	remaining: 10.1s
666:	learn: 11922.0033486	total: 8.04s	remaining: 10

805:	learn: 11184.0924992	total: 9.76s	remaining: 8.4s
806:	learn: 11162.6829038	total: 9.78s	remaining: 8.39s
807:	learn: 11158.6613165	total: 9.79s	remaining: 8.38s
808:	learn: 11153.9474510	total: 9.8s	remaining: 8.37s
809:	learn: 11146.8579688	total: 9.81s	remaining: 8.36s
810:	learn: 11143.0457890	total: 9.83s	remaining: 8.36s
811:	learn: 11138.0385783	total: 9.85s	remaining: 8.35s
812:	learn: 11132.0174038	total: 9.87s	remaining: 8.34s
813:	learn: 11129.4497442	total: 9.88s	remaining: 8.32s
814:	learn: 11116.6829892	total: 9.89s	remaining: 8.31s
815:	learn: 11113.8640946	total: 9.9s	remaining: 8.3s
816:	learn: 11106.3651905	total: 9.91s	remaining: 8.28s
817:	learn: 11100.0090474	total: 9.92s	remaining: 8.27s
818:	learn: 11097.6734910	total: 9.93s	remaining: 8.25s
819:	learn: 11096.4635249	total: 9.93s	remaining: 8.24s
820:	learn: 11092.6996363	total: 9.94s	remaining: 8.22s
821:	learn: 11089.1976930	total: 9.95s	remaining: 8.21s
822:	learn: 11085.3376263	total: 9.96s	remaining: 8.

965:	learn: 10491.4724320	total: 12s	remaining: 6.63s
966:	learn: 10486.4361963	total: 12s	remaining: 6.62s
967:	learn: 10484.2808737	total: 12s	remaining: 6.61s
968:	learn: 10480.7839673	total: 12s	remaining: 6.6s
969:	learn: 10478.3396135	total: 12.1s	remaining: 6.59s
970:	learn: 10477.0121242	total: 12.1s	remaining: 6.58s
971:	learn: 10474.3085486	total: 12.1s	remaining: 6.57s
972:	learn: 10472.4694097	total: 12.1s	remaining: 6.56s
973:	learn: 10469.1072653	total: 12.1s	remaining: 6.55s
974:	learn: 10449.5936531	total: 12.1s	remaining: 6.54s
975:	learn: 10445.8989626	total: 12.2s	remaining: 6.53s
976:	learn: 10443.3533408	total: 12.2s	remaining: 6.51s
977:	learn: 10435.6259861	total: 12.2s	remaining: 6.5s
978:	learn: 10434.0530298	total: 12.2s	remaining: 6.49s
979:	learn: 10431.2670510	total: 12.2s	remaining: 6.48s
980:	learn: 10428.8299918	total: 12.2s	remaining: 6.47s
981:	learn: 10424.3597183	total: 12.3s	remaining: 6.46s
982:	learn: 10422.8462346	total: 12.3s	remaining: 6.45s
98

1120:	learn: 9987.8351392	total: 14s	remaining: 4.74s
1121:	learn: 9986.4960392	total: 14s	remaining: 4.73s
1122:	learn: 9984.9432007	total: 14.1s	remaining: 4.72s
1123:	learn: 9984.2859518	total: 14.1s	remaining: 4.71s
1124:	learn: 9982.1306948	total: 14.1s	remaining: 4.7s
1125:	learn: 9977.3723045	total: 14.1s	remaining: 4.68s
1126:	learn: 9976.7310714	total: 14.1s	remaining: 4.67s
1127:	learn: 9975.6339155	total: 14.1s	remaining: 4.66s
1128:	learn: 9973.6567076	total: 14.1s	remaining: 4.65s
1129:	learn: 9972.1192910	total: 14.2s	remaining: 4.63s
1130:	learn: 9968.5360268	total: 14.2s	remaining: 4.62s
1131:	learn: 9966.1870100	total: 14.2s	remaining: 4.61s
1132:	learn: 9963.3667169	total: 14.2s	remaining: 4.6s
1133:	learn: 9955.6578380	total: 14.2s	remaining: 4.58s
1134:	learn: 9954.9184189	total: 14.2s	remaining: 4.57s
1135:	learn: 9954.3414942	total: 14.2s	remaining: 4.56s
1136:	learn: 9953.1187060	total: 14.2s	remaining: 4.55s
1137:	learn: 9951.8991133	total: 14.3s	remaining: 4.54

1282:	learn: 9532.5252938	total: 16.1s	remaining: 2.71s
1283:	learn: 9529.7205519	total: 16.1s	remaining: 2.7s
1284:	learn: 9528.6706150	total: 16.1s	remaining: 2.69s
1285:	learn: 9525.4219021	total: 16.1s	remaining: 2.68s
1286:	learn: 9522.8473390	total: 16.1s	remaining: 2.67s
1287:	learn: 9521.6785760	total: 16.1s	remaining: 2.65s
1288:	learn: 9518.8688755	total: 16.1s	remaining: 2.64s
1289:	learn: 9517.4965093	total: 16.1s	remaining: 2.63s
1290:	learn: 9515.4801653	total: 16.2s	remaining: 2.62s
1291:	learn: 9508.9837898	total: 16.2s	remaining: 2.6s
1292:	learn: 9507.6234909	total: 16.2s	remaining: 2.59s
1293:	learn: 9495.4093049	total: 16.2s	remaining: 2.58s
1294:	learn: 9493.4576861	total: 16.2s	remaining: 2.56s
1295:	learn: 9492.6548251	total: 16.2s	remaining: 2.55s
1296:	learn: 9489.5324710	total: 16.2s	remaining: 2.54s
1297:	learn: 9488.7373630	total: 16.2s	remaining: 2.53s
1298:	learn: 9487.2508412	total: 16.3s	remaining: 2.52s
1299:	learn: 9485.6052083	total: 16.3s	remaining: 

1442:	learn: 9113.1625095	total: 18.3s	remaining: 723ms
1443:	learn: 9111.9892718	total: 18.3s	remaining: 710ms
1444:	learn: 9109.8430179	total: 18.3s	remaining: 698ms
1445:	learn: 9096.9996876	total: 18.3s	remaining: 685ms
1446:	learn: 9096.3037066	total: 18.4s	remaining: 672ms
1447:	learn: 9095.0491094	total: 18.4s	remaining: 660ms
1448:	learn: 9093.9255018	total: 18.4s	remaining: 647ms
1449:	learn: 9093.3939852	total: 18.4s	remaining: 634ms
1450:	learn: 9092.5328587	total: 18.4s	remaining: 622ms
1451:	learn: 9091.2675612	total: 18.4s	remaining: 609ms
1452:	learn: 9090.4426371	total: 18.4s	remaining: 596ms
1453:	learn: 9089.0539828	total: 18.4s	remaining: 583ms
1454:	learn: 9088.7025184	total: 18.5s	remaining: 571ms
1455:	learn: 9087.2592361	total: 18.5s	remaining: 558ms
1456:	learn: 9085.7069642	total: 18.5s	remaining: 545ms
1457:	learn: 9084.8957304	total: 18.5s	remaining: 533ms
1458:	learn: 9083.6475975	total: 18.5s	remaining: 520ms
1459:	learn: 9083.2738438	total: 18.5s	remaining

In [34]:
tof = prep_for_prediction(test_official, istrain=False)

In [35]:
my_info(tof)

,Column,Data Type,NaN Count,NaN Percentage,Mean,Minimum,Maximum,Median
0,city,object,0,0.0%,NaN,NaN,NaN,NaN
1,floor,object,0,0.0%,NaN,NaN,NaN,NaN
2,id,object,0,0.0%,NaN,NaN,NaN,NaN
3,reform_count_of_houses_1000,int64,0,0.0%,153.5669,0,649,130.0
4,reform_count_of_houses_500,int64,0,0.0%,45.2942,0,213,39.0
5,reform_house_population_1000,float64,0,0.0%,2621.5376,1.0,8242.0,2407.5
6,reform_house_population_500,float64,0,0.0%,781.5121,1.0,2893.0,740.0
7,reform_mean_year_building_1000,float64,0,0.0%,1964.7577,1625.3041,2015.5833,1969.8587
8,reform_mean_year_building_500,float64,0,0.0%,1965.4738,1403.2143,2020.0,1970.5973
9,region,object,0,0.0%,NaN,NaN,NaN,NaN


In [37]:
test_pool_official = Pool(tof
                 ,cat_features=get_cat_vars(tof)
                 )

In [38]:
preds_official = model.predict(test_pool_official)

In [40]:
for_export_test_official = test_official.copy()
for_export_test_official['per_square_meter_price'] = preds_official
ans_catboost_full_osm_united_dropped = for_export_test_official[['id','per_square_meter_price']]
csv_export(ans_catboost_full_osm_united_dropped)

In [39]:
len(preds_official)

2974

In [150]:
ans_catboost_more_p1_cleaner

,id,per_square_meter_price
0,COL_289284,46461.7383
1,COL_289305,47872.3180
2,COL_289318,43325.0216
3,COL_289354,99938.4964
4,COL_289399,53159.3638
5,COL_289428,45073.8344
6,COL_289460,44909.1118
7,COL_289493,60337.2923
8,COL_289495,53557.1845
9,COL_289749,50434.1706


In [55]:
ans_catboost_more_p1

,id,per_square_meter_price
0,COL_289284,42838.7838
1,COL_289305,45622.6530
2,COL_289318,38460.1682
3,COL_289354,98991.3012
4,COL_289399,48360.6069
5,COL_289428,41007.3140
6,COL_289460,55891.2751
7,COL_289493,61761.2181
8,COL_289495,54257.1453
9,COL_289749,48265.0149


In [182]:
tof = test_official.copy()

In [184]:
tof

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type,floors
0,Курск,1,COL_289284,7,55,85,117,0,0,0,0,0,2,6,10,4.1017,Курск,4.4321e+05,0.1471,0,7,7,19,0,0,0,0,1,2,3,3,5,5,5,2,2,4,7,7,8,0,0,1,0,0,0,1,5,36,61,83,435.3635,5.6368,0,0,0,0.1087,11,15,27,184,78,1997.0,743.0,4.3250,4.2113,1966.4716,1966.7403,Курская область,51.7093,36.1479,156.1490,S6983,2020-09-06,100,1,1
1,Сургут,1,COL_289305,8,70,112,140,0,0,0,0,0,7,8,13,4.0842,Сургут,3.8063e+05,0.1625,0,18,33,50,0,0,0,1,1,2,5,6,3,6,8,1,1,1,3,5,5,3,6,7,0,3,5,8,4,51,81,99,693.4479,9.6416,0,0,0,0.0914,9,11,19,118,44,2823.0,1019.0,5.3898,5.5000,1988.2593,1989.0682,Ханты-Мансийский АО,61.2332,73.4625,190.7379,S29120,2020-09-06,110,1,1
2,Тюмень,-1,COL_289318,3,28,67,122,0,0,0,0,0,9,18,34,1.2920,Тюмень,7.4455e+05,0.0758,1,21,57,92,0,0,1,2,0,1,4,5,2,3,3,1,2,5,0,1,1,0,2,5,0,1,1,3,3,15,34,63,298.9047,1.8040,0,0,0,0.1203,6,20,29,150,51,3484.0,1332.0,7.9155,8.2500,1985.8803,1991.4583,Тюменская область,57.1431,65.5546,457.1181,S23731,2020-09-06,10,1,1
3,Иркутск,1,COL_289354,5,76,139,231,0,0,0,0,0,28,52,84,0.9146,Иркутск,6.2356e+05,0.0436,1,30,60,95,1,8,13,18,0,5,6,10,5,8,12,12,20,32,1,8,18,1,5,5,0,7,11,16,3,14,26,51,60.2247,1.3381,0,0,0,0.0793,9,18,29,252,86,1975.0,666.0,3.2769,3.0120,1947.0733,1941.6579,Иркутская область,52.2814,104.2830,66.5036,S14207,2020-09-06,100,1,1
4,Курск,1,COL_289399,8,105,189,279,0,0,2,9,1,24,40,54,1.3001,Курск,4.4321e+05,0.0842,1,40,72,84,1,10,12,12,3,10,14,21,2,6,6,10,14,17,1,1,1,4,5,12,0,2,12,16,1,49,98,156,432.5301,2.4410,0,0,0,0.0936,12,26,37,114,31,1044.0,394.0,4.3462,4.8276,1948.7642,1946.6897,Курская область,51.7297,36.1940,23.8649,S20658,2020-09-06,10,1,0
5,Иркутск,1,COL_289428,0,18,30,33,0,0,0,0,0,6,9,9,5.4943,Иркутск,6.2356e+05,0.1410,0,13,15,17,0,0,0,0,0,0,0,0,2,3,3,0,0,0,1,2,2,0,0,0,0,2,2,2,0,8,14,16,58.4055,6.7825,0,0,0,0.1867,7,12,12,62,27,489.0,260.0,2.9344,3.1852,1963.6066,1968.9231,Иркутская область,52.2763,104.3612,58.3778,S14771,2020-09-06,10,1,1
6,Кемерово,1,COL_289460,1,5,7,20,0,0,0,0,0,0,0,0,6.4769,Кемерово,5.5866e+05,0.5237,0,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,1,5,5,12,206.3094,3.45